In [ ]:
from fastai.vision import *

In [ ]:
path = Path('../data/histopathologic/')

In [ ]:
path.ls()

In [ ]:
len((path/'train').ls()), len((path/'test').ls())

In [ ]:
tfms = get_transforms(True, True, max_rotate=20, max_zoom=1.1, max_lighting=0.2, max_warp=0,
                     p_affine=0.5, p_lighting=0.5)

data = ImageDataBunch.from_csv(path=path, folder='train', valid_pct=0.2,
                               csv_labels='train_labels.csv', suffix='.tif',
                               ds_tfms=tfms,
                               size=96, bs=128)

In [ ]:
data.add_test(ImageList.from_folder(path/'test'))

In [ ]:
data.valid_ds.y.items.mean(), data.train_ds.y.items.mean()

In [ ]:
data.show_batch(3)

In [ ]:
from sklearn.metrics import roc_auc_score
class AUC(Callback):
    "AUC score"
    
    def on_epoch_begin(self, **kwargs): 
        self.outputs = []
        self.targets = []

    def on_batch_end(self, last_output, last_target, **kwargs):
        self.outputs += list(to_np(last_output)[:, 1])
        self.targets += list(to_np(last_target))

    def on_epoch_end(self, last_metrics, **kwargs): 
        return {'last_metrics': last_metrics + [roc_auc_score(self.targets, self.outputs)]}

In [ ]:
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy)
learn.lr_find()
auc = AUC()
learn.metrics = [accuracy, auc]

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 1e-2
learn.fit_one_cycle(3,max_lr=lr)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True, k=5)

In [ ]:
learn.get_preds(ds_type=DatasetType)

In [ ]:
learn.fit_one_cycle(1, max_lr=slice(3e-5))

In [ ]:
test_preds = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
probas, _ = test_preds

In [ ]:
sample_submission = pd.read_csv(path/'sample_submission.csv')

In [ ]:
labels = to_np(torch.argmax(probas, 1))
names = [o.name.split('.')[0] for o in data.test_ds.items]

In [ ]:
sample_submission['label'] = sample_submission['id'].map(dict(zip(names, labels)))

In [ ]:
sample_submission.to_csv("vgg16_bn_first_sub.csv", index=False)

In [ ]:
!kaggle competitions submit -c histopathologic-cancer-detection -f vgg16_bn_first_sub.csv -m "init"